In [1]:
from src.imgModifier import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
from PIL import Image
import scipy.signal
import glob
import pickle
import cv2
import os

## Creamos el dataframe con las imagenes, labels, y paths 

In [27]:
labels_t = ['10c','50c','20c','5c','2c','1e','1c','2e']
origin_path = "./Inputs/coins"

In [28]:
imagenes,img_label,img_path = readImages(origin_path,labels_t)

In [29]:
coins_original = pd.DataFrame(list(zip(img_label, imagenes, img_path)), columns =['label', 'image', 'path'])
print(coins_original.shape)
coins_original.head()

(2200, 3)


,label,image,path
0,10c,"[[[19, 68, 100], [52, 111, 143], [150, 220, 25...",Inputs/coins/10c/train_21_56.jpg
1,10c,"[[[221, 223, 223], [221, 223, 223], [221, 223,...",Inputs/coins/10c/test_18_24.jpg
2,10c,"[[[65, 97, 132], [51, 83, 118], [43, 74, 107],...",Inputs/coins/10c/IMG_4207_17.jpg
3,10c,"[[[1, 6, 27], [0, 7, 27], [2, 7, 32], [4, 13, ...",Inputs/coins/10c/IMG_4187_42.jpg
4,10c,"[[[190, 192, 192], [191, 193, 193], [192, 194,...",Inputs/coins/10c/train_8_4.jpg


In [30]:
print(coins_original['label'].unique())
print(coins_original.shape)
print(coins_original['label'].value_counts())

['10c' '50c' '20c' '5c' '2c' '1e' '1c' '2e']
(2200, 3)
1e     467
5c     364
2e     344
10c    307
20c    272
2c     163
1c     155
50c    128
Name: label, dtype: int64


## Creamos segundo df con los arrays (aplicando el resize (70x70) y haciendo np.stack(img)/255) de las imagenes y sus labels 

In [31]:
coins = pd.DataFrame()

In [ ]:
coins['image'] = coins_original['image'].apply(transfImg)

In [33]:
coins = coins.join(coins_original['label'])

In [34]:
display(coins.head())
print(coins.shape)


,image,label
0,"[[[0.10588235294117647, 0.3176470588235294, 0....",10c
1,"[[[0.8901960784313725, 0.8980392156862745, 0.8...",10c
2,"[[[0.23529411764705882, 0.3607843137254902, 0....",10c
3,"[[[0.0, 0.0196078431372549, 0.1019607843137254...",10c
4,"[[[0.7490196078431373, 0.7568627450980392, 0.7...",10c


(2200, 2)


## Creamos más dfs con los arrais rotados para tener más datos  

In [9]:
# Rota los arrays x grados
def rotateConcat(coins_original, coins, grados):
    coins_gr = pd.DataFrame()
    coins_gr['image'] = coins_original['image'].apply(lambda x: rotAndTransfImg(x,grados))
    coins_gr = coins_gr.join(coins_original['label'])
    coins = pd.concat([coins,coins_gr])
    print(coins.shape)
    return coins

In [60]:
# Al tener mas imagenes de un tipo que de otro hemos rotado mas veces algunos arrays 
for g in range(10,360,30):
    coins_gr = pd.DataFrame()
    coins_gr['image'] = coins_original[coins_original['label']=='50c']['image'].apply(lambda x: rotAndTransfImg(x,g))
    coins_gr = coins_gr.join(coins_original[coins_original['label']=='50c']['label'])
    coins = pd.concat([coins,coins_gr])
    print(coins.shape)


(96562, 2)
(96690, 2)
(96818, 2)
(96946, 2)
(97074, 2)
(97202, 2)
(97330, 2)
(97458, 2)
(97586, 2)
(97714, 2)
(97842, 2)
(97970, 2)


In [61]:
# finalmente tenemos un df con mas 10.000 'imagenes'
print(coins['label'].unique())
print(coins.shape)
print(coins['label'].value_counts())

['10c' '50c' '20c' '5c' '2c' '1e' '1c' '2e']
(97970, 2)
20c    13872
2e     13760
5c     13104
10c    12894
2c     11247
1e     11208
1c     11005
50c    10880
Name: label, dtype: int64


In [43]:
grados = [e for e in range(15,360,15)]
len(grados)

23

In [42]:
# Aplicamos la fn para rotar y concatenar los arrays al df
for grado in grados:
    coins = rotateConcat(coins_original, coins, grado)

(18706, 2)
(20906, 2)
(23106, 2)
(25306, 2)
(27506, 2)
(29706, 2)
(31906, 2)
(34106, 2)
(36306, 2)
(38506, 2)
(40706, 2)
(42906, 2)
(45106, 2)
(47306, 2)
(49506, 2)
(51706, 2)
(53906, 2)
(56106, 2)
(58306, 2)
(60506, 2)
(62706, 2)
(64906, 2)
(67106, 2)


In [62]:
# Comprobamos que todos los arrays sean de 70x70x3
for im in coins['image']:
    if im.shape != (70, 70, 3):
        print('¡ojo!', im.shape)

In [64]:
# Guardamos nuestro df en un archivo pickle:

coins.to_pickle('./Output/coins_97970_arrays_label.pkl')  

In [65]:
coins_4lab = coins.copy()